In [1]:
from projection_coeffs import Density_Projection_Calculator

# Generic imports
import numpy as np
import time
from ase.io import read
from helpers.helpers import filter_by_status
from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species,mask_center_atoms_by_id
from rascal.utils import (spherical_expansion_reshape, lm_slice)
from loader.loader import load_data
from joblib import Parallel, delayed, parallel_backend
from helpers.helpers import grouper

In [2]:
hypers_lode = {'smearing':1.0, # WARNING: comp. cost scales cubically with 1/smearing
        'max_angular':6,
        'max_radial': 1,       
        'cutoff_radius':4.5,
        'potential_exponent':1,
        'compute_gradients': False
              }

frames = read('../make_tensor_data/train_tensor/CSD-3k+S546_shift_tensors.xyz',':')
frames = filter_by_status(frames, status="PASSING")
for frame in frames: frame.wrap(eps=1e-12)

In [2]:
train_structures, test_structures, train_properties, test_properties = load_data("../make_tensor_data/train_tensor/CSD-3k+S546_shift_tensors.xyz",\
                                                                                    "../make_tensor_data/test_tensor/CSD-500+104-7_shift_tensors.xyz")

In [12]:
train_structures[:5]

[Atoms(symbols='N4O4C20H20', pbc=True, cell=[13.564047596, 5.79502231063, 5.60401339238], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C18H36N8O8', pbc=True, cell=[[7.131018008, 0.0, 0.0], [-3.35710500272, 9.26783468512, 0.0], [-2.0474364974, -1.27908356908, 9.61766816655]], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C28H16N32O32', pbc=True, cell=[[12.722757232, 0.0, 0.0], [0.0, 11.8045558426, 0.0], [-0.879600543992, 0.0, 8.09690265277]], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C64H118N14O4', pbc=True, cell=[[6.71344207, 0.0, 0.0], [-0.499359248051, 7.71360395615, 0.0], [-1.05722614374, -2.25817365532, 31.890877292]], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C32H32N16O16', pbc=True, cell=[[13.24907966, 0.0, 0.0], [0.0, 11.1910736183, 0.0], [-0.519960131257, 0.0, 6.25747407802]], cs_iso=..., cs_tensor=...)]

In [13]:
frames[:5]

[Atoms(symbols='N4O4C20H20', pbc=True, cell=[13.564047596, 5.79502231063, 5.60401339238], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C18H36N8O8', pbc=True, cell=[[7.131018008, 0.0, 0.0], [-3.35710500272, 9.26783468512, 0.0], [-2.0474364974, -1.27908356908, 9.61766816655]], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C28H16N32O32', pbc=True, cell=[[12.722757232, 0.0, 0.0], [0.0, 11.8045558426, 0.0], [-0.879600543992, 0.0, 8.09690265277]], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C64H118N14O4', pbc=True, cell=[[6.71344207, 0.0, 0.0], [-0.499359248051, 7.71360395615, 0.0], [-1.05722614374, -2.25817365532, 31.890877292]], cs_iso=..., cs_tensor=...),
 Atoms(symbols='C32H32N16O16', pbc=True, cell=[[13.24907966, 0.0, 0.0], [0.0, 11.1910736183, 0.0], [-0.519960131257, 0.0, 6.25747407802]], cs_iso=..., cs_tensor=...)]

In [4]:


hypers_2 = {
    'interaction_cutoff': 2,
    'max_radial': 6,
    'max_angular': 6,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.2,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO',
    'normalize' : False,
    'soap_type' : "PowerSpectrum"
}

species_dict = {'H': 0, 'C':1, 'N':2, 'O':3, 'S':4}

In [4]:
import time

In [5]:
lode = Density_Projection_Calculator(**hypers_lode)

In [28]:
start_time = time.time()
get_lode_in_parallel(frames[:80], hypers_lode,species_dict,blocksize=5,n_cores=-1)
print("--- 20 structures took %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [26]:
def get_lode(frames,calculator,hypers,species_dict):
    calculatorinstance = calculator(**hypers)
    #print("worker spawned")
    return calculatorinstance.transform(frames,species_dict).get_features(calculatorinstance)



def get_lode_in_parallel(frames,hypers,species_dict,blocksize=5,n_cores=-1):
    """helper function that returns the features of a calculator (from calculator.transform())
       in parallel
    """
    
    #block is necessary to ensure that shape of the chunks is equal
    #replace by get_atomic_species functions
    

    with parallel_backend(backend="multiprocessing"):
        results = Parallel(n_jobs=n_cores)(delayed(get_lode)(frame, Density_Projection_Calculator, hypers, species_dict) for frame in grouper(blocksize,frames))
    
    return np.concatenate(results)

In [27]:
feat3 = lode.get_features()

In [28]:
feat3.shape

(2250, 5, 1, 49)

In [5]:
feat3 = feat3.reshape(feat3.shape[0],-1)

NameError: name 'feat3' is not defined

In [6]:
spex = SphericalExpansion(**dict(filter(lambda el: el[0] not in ['normalize', 'soap_type'], hypers_2.items()) ) )

In [8]:
feat4 = spherical_expansion_reshape(spex.transform(train_structures).get_features(spex), **hypers_2)

In [11]:
feat4.reshape((feat4.shape[0],-1)).shape

(330082, 1470)

In [31]:
feat5 = np.moveaxis(np.asarray(
    [np.einsum("Aanm,Abqm->Aabnq", feat3[...,lm_slice(l)], feat4[...,lm_slice(l)]) for l in range(hypers_lode["max_angular"]+1) ]), 0, -1)
feat5 = feat5.reshape(len(feat5), -1)

In [32]:
feat5.shape

(2250, 840)

In [19]:
np.sum([48, 70, 108, 200, 96, 86, 136, 196, 108, 144, 156, 22, 144, 104, 96, 100, 132, 176, 96, 32])

2250

In [10]:
feats = run_example_shiftML()


ShiftML data set
Number of structures         =  3546
Total # of environments      =  340941
Average # of atoms per frame =  96.1
Number of atoms of first 20 structures =  [48, 70, 108, 200, 96, 86, 136, 196, 108, 144, 156, 22, 144, 104, 96, 100, 132, 176, 96, 32]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [29]:
import numpy as np

In [30]:
np.sqrt(19.)

4.358898943540674